In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import pandas as pd

2024-04-11 00:28:55.963530: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 00:28:55.966094: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 00:28:56.036381: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 00:28:56.352580: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 00:28:58.631386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [33]:
! pip install pandas


Defaulting to user installation because normal site-packages is not writeable


In [2]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [3]:
timeseries_data_global= pd.read_csv("../Dataset/global_cases.csv")
timeseries_data_india= pd.read_csv("../Dataset/india_cases.csv")

In [4]:
timeseries_data_global.head()

,Date,Cases
0,22/01/20,557
1,23/01/20,657
2,24/01/20,944
3,25/01/20,1437
4,26/01/20,2120


In [5]:
timeseries_data_india.head()

,Date,Cases
0,22/01/20,0
1,23/01/20,0
2,24/01/20,0
3,25/01/20,0
4,26/01/20,0


In [6]:
global_cases=timeseries_data_global['Cases']


In [7]:
global_cases_array=np.array(global_cases)

In [8]:
global_cases_array

array([      557,       657,       944, ..., 676213378, 676392824,
       676570149])

In [9]:
india_cases=timeseries_data_india['Cases']

In [10]:
india_cases_array=np.array(india_cases)

In [11]:
india_cases_array

array([       0,        0,        0, ..., 44689919, 44690298, 44690738])

In [12]:
timestamps=10
X_prevdata_global,y_preddata_global=prepare_data(global_cases_array, timestamps)

In [16]:
print(X_prevdata_global)
X_prevdata_global.shape

[[      557       657       944 ...      6169      8237      9927]
 [      657       944      1437 ...      8237      9927     12038]
 [      944      1437      2120 ...      9927     12038     16787]
 ...
 [674978793 675044414 675171439 ... 675968775 676024901 676082941]
 [675044414 675171439 675322238 ... 676024901 676082941 676213378]
 [675171439 675322238 675542852 ... 676082941 676213378 676392824]]


(1133, 10)

In [14]:
print(y_preddata_global)

[    12038     16787     19887 ... 676213378 676392824 676570149]


In [18]:
n_feature=1
X_prevdata_global=X_prevdata_global.reshape((X_prevdata_global.shape[0],X_prevdata_global.shape[1],n_feature))

In [19]:
X_prevdata_global=np.array(X_prevdata_global)

In [20]:
X_prevdata_global.shape

(1133, 10, 1)

In [21]:
model = Sequential()
model.add(LSTM(65, activation='relu', return_sequences=True, input_shape=(timestamps, n_feature)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_prevdata_global,y_preddata_global, epochs=100, verbose=1)

/home/soumesh/miniconda3/envs/testenv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 116320453248679936.0000
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 105546519196729344.0000
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 93738889595846656.0000
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 93996999950467072.0000
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 94197952880312320.0000
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 93979467893964800.0000
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 92351202842378240.0000
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 89455510121611264.0000
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 91008578885779456.0000
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 91199576081432576.0000
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 88055806049648640.0000
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 92165445506826240.0000
Epoch 13/1

IndexError: tuple index out of range